# PYTORCH CNN Classifier

To run this notebook on an another benchmark, use

```
papermill utils/torch_cnn_classifier.ipynb torch_cnn_experiments/[DATASET NAME].ipynb -p DATASET [DATASET NAME]
```

In [1]:
# DATASET = 'no_dataset'
DATASET = 'demo_human_or_worm'
VERSION = 0
BATCH_SIZE = 64
EPOCHS = 1

In [2]:
# Parameters
DATASET = "human_enhancers_cohn"
EPOCHS = 10


In [3]:
print(DATASET, VERSION, BATCH_SIZE, EPOCHS)

human_enhancers_cohn 0 64 10


## Config

In [4]:
import os
import numpy as np
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchtext.data.utils import get_tokenizer

from genomic_benchmarks.data_check import is_downloaded, info
from genomic_benchmarks.dataset_getters.pytorch_datasets import get_dataset
from genomic_benchmarks.loc2seq import download_dataset
from genomic_benchmarks.models.torch import CNN
from genomic_benchmarks.dataset_getters.utils import coll_factory, LetterTokenizer, build_vocab, check_seq_lengths, check_config, VARIABLE_LENGTH_DATASETS

/opt/conda/envs/bench_env/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
USE_PADDING = DATASET in VARIABLE_LENGTH_DATASETS

## Choose the dataset

In [6]:
if not is_downloaded(DATASET):
    download_dataset(DATASET, local_repo=True)

/home/jovyan/genomic_benchmarks/src/genomic_benchmarks/utils/datasets.py:50: UserWarning: No version specified. Using version 0.
  warnings.warn(f"No version specified. Using version {metadata['version']}.")
Downloading...
From: https://drive.google.com/uc?id=176563cDPQ5Y094WyoSBF02QjoVQhWuCh
To: /home/jovyan/.genomic_benchmarks/human_enhancers_cohn.zip
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 11.9M/11.9M [00:00<00:00, 44.9MB/s]


In [7]:
info(DATASET, local_repo=True)

Dataset `human_enhancers_cohn` has 2 classes: negative, positive.

All lengths of genomic intervals equals 500.

Totally 27791 sequences have been found, 20843 for training and 6948 for testing.


,train,test
negative,10422,3474
positive,10421,3474


In [8]:
train_dset = get_dataset(DATASET, 'train')

In [9]:
NUM_CLASSES = len(set([train_dset[i][1] for i in range(len(train_dset))]))
NUM_CLASSES

2

## Tokenizer and vocab

In [10]:
tokenizer = get_tokenizer(LetterTokenizer())
vocabulary = build_vocab(train_dset, tokenizer, use_padding=USE_PADDING)

print("vocab len:" ,vocabulary.__len__())
print(vocabulary.get_stoi())

vocab len: 7
{'<eos>': 6, 'G': 5, 'A': 4, 'C': 3, 'T': 2, '<bos>': 1, '<unk>': 0}


## Dataloader and batch preparation

In [11]:
# Run on GPU or CPU
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

max_seq_len, nn_input_len = check_seq_lengths(dataset=train_dset, use_padding=USE_PADDING)

# Data Loader
if(USE_PADDING):
    collate = coll_factory(vocabulary, tokenizer, device, pad_to_length = nn_input_len)
else:
    collate = coll_factory(vocabulary, tokenizer, device, pad_to_length = None)

train_loader = DataLoader(train_dset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate)

Using cuda device
max_seq_len  500


## Model

In [12]:
model = CNN(
    number_of_classes=NUM_CLASSES,
    vocab_size=vocabulary.__len__(),
    embedding_dim=100,
    input_len=nn_input_len,
    device=device
).to(device)

## Training

In [13]:
model.fit(train_loader, epochs=EPOCHS)

Epoch 0
Train metrics: 
 Accuracy: 64.6%, Avg loss: 0.653962 

Epoch 1
Train metrics: 
 Accuracy: 67.5%, Avg loss: 0.643216 

Epoch 2
Train metrics: 
 Accuracy: 68.7%, Avg loss: 0.639047 

Epoch 3
Train metrics: 
 Accuracy: 68.9%, Avg loss: 0.637780 

Epoch 4
Train metrics: 
 Accuracy: 69.6%, Avg loss: 0.635052 

Epoch 5
Train metrics: 
 Accuracy: 70.1%, Avg loss: 0.632791 

Epoch 6
Train metrics: 
 Accuracy: 70.8%, Avg loss: 0.631207 

Epoch 7
Train metrics: 
 Accuracy: 70.5%, Avg loss: 0.630917 

Epoch 8
Train metrics: 
 Accuracy: 70.9%, Avg loss: 0.629024 

Epoch 9
Train metrics: 
 Accuracy: 71.6%, Avg loss: 0.626794 



## Testing

In [14]:
test_dset = get_dataset(DATASET, 'test')
test_loader = DataLoader(test_dset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate)

acc, f1 = model.test(test_loader)
acc, f1

Test metrics: 
 Accuracy: 0.694588, F1 score: 0.670804, Avg loss: 0.644409 



(0.6945883707541739, 0.6708035991312443)